In [3]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
import hydra
import os
from pathlib import Path
import xgboost as xgb


from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import yaml
# local imports
from hydra import compose, initialize
from src.data_processing.preprocessing.pandas_preprocessors import *
from src.data_processing.pipelines.ClassifierPipe import ClassifierPipe




In [3]:
with open('/Users/mds8301/Development/ENIGMA/modeling/experiments/conf/local_config.yaml', 'r') as f:
    config = yaml.safe_load(f)
    
DATA_PATH = config['local_config']['data_path']

In [4]:
pd.read_parquet(DATA_PATH)


,mouse_id,day,event,sensor,time,trial,signal,action,latency,sex,learning_phase,trial_count
0,12,5,cue,D2,-25.000000,0,-0.155359,escape,0.00,F,0,115
1,12,5,cue,D2,-24.901531,0,-0.420553,escape,0.00,F,0,115
2,12,5,cue,D2,-24.803064,0,-1.592294,escape,0.00,F,0,115
3,12,5,cue,D2,-24.704596,0,-1.268734,escape,0.00,F,0,115
4,12,5,cue,D2,-24.606127,0,-0.210176,escape,0.00,F,0,115
...,...,...,...,...,...,...,...,...,...,...,...,...
5860105,1,4,escape,D1,19.606127,1,0.299603,avoid,7.54,F,0,87
5860106,1,4,escape,D1,19.704596,1,-0.249408,avoid,7.54,F,0,87
5860107,1,4,escape,D1,19.803064,1,-0.486369,avoid,7.54,F,0,87
5860108,1,4,escape,D1,19.901531,1,-0.146374,avoid,7.54,F,0,87


In [15]:
def df_pipeline(df):
    '''pandas preprocessing specific to this experiment'''
    drop_columns = ["action", "sex", "trial_count", "trial"]
    df_ = (
        df
        .query("sensor=='DA' or sensor == 'D'")
        .pipe(calculate_max_min_signal)
        .pipe(calculate_percent_avoid)
        .drop(columns=drop_columns)
        .pipe(expand_df)
        .drop(columns=['mouse_id', 'day'])
    )
    return df_

PROCESSOR_PIPE = (ClassifierPipe(DATA_PATH)
                    .read_raw_data()
                    .pandas_pipe(df_pipeline)
                    # .split_by_ratio(target='ratio_avoid')
                    # .transform_data()
                    )
# ids = assign_ids(df)
PROCESSOR_PIPE.processed_data


,ratio_avoid,neg_signal_trapz_D2_avoid,neg_signal_trapz_D2_cue,neg_signal_trapz_D2_escape,neg_signal_trapz_D2_shock,neg_signal_trapz_DA_avoid,neg_signal_trapz_DA_cue,neg_signal_trapz_DA_escape,neg_signal_trapz_DA_shock,pos_signal_trapz_D2_avoid,...,signal_min_DA_escape,signal_min_DA_shock,signal_trapz_D2_avoid,signal_trapz_D2_cue,signal_trapz_D2_escape,signal_trapz_D2_shock,signal_trapz_DA_avoid,signal_trapz_DA_cue,signal_trapz_DA_escape,signal_trapz_DA_shock
0,0.0,NaN,NaN,NaN,NaN,-175.680374,-167.158646,-171.241150,-174.943573,NaN,...,-2.351609,-2.389435,NaN,NaN,NaN,NaN,-15.749731,-1.979335,1.762733,-3.865334
1,0.0,NaN,NaN,NaN,NaN,-162.030914,-166.663467,-181.359558,-180.819473,NaN,...,-1.758063,-1.758867,NaN,NaN,NaN,NaN,19.392359,0.428599,-17.635229,-17.589060
2,0.0,NaN,NaN,NaN,NaN,-188.438995,-188.475937,-182.854126,-181.354370,NaN,...,-2.447979,-2.518370,NaN,NaN,NaN,NaN,-9.620571,-6.473549,42.737228,47.267506
3,0.0,NaN,NaN,NaN,NaN,-187.598328,-182.339111,-217.391159,-198.629608,NaN,...,-2.808737,-2.584502,NaN,NaN,NaN,NaN,-34.904079,-22.436991,55.949619,90.593544
4,0.0,NaN,NaN,NaN,NaN,-156.896820,-164.152283,-235.104416,-247.854660,NaN,...,-2.159111,-2.210720,NaN,NaN,NaN,NaN,35.344769,35.023357,-7.665092,-30.157568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,1.0,NaN,NaN,NaN,NaN,-183.375275,-180.726212,-252.993683,-252.885529,NaN,...,-2.395465,-2.416087,NaN,NaN,NaN,NaN,-1.635001,3.024579,-116.535851,-115.549271
92,1.0,-149.051620,-156.074051,-184.906982,-181.503662,-192.717606,-184.584946,-268.689178,-270.515015,182.944382,...,-2.086705,-2.104770,33.892754,21.481998,-23.793985,-18.835018,-12.810680,-7.558132,-146.258148,-148.969711
93,1.0,-125.662460,-135.946121,-238.629349,-239.908524,-215.769089,-207.240372,-261.838837,-248.177307,168.404358,...,-1.979279,-1.932195,42.741905,17.755304,-147.939438,-147.215317,-45.969501,-31.608734,-141.356522,-120.681953
94,1.0,-114.695328,-121.647949,NaN,NaN,-190.794174,-195.882523,NaN,NaN,145.594589,...,NaN,NaN,30.899246,17.841713,NaN,NaN,3.344767,-7.479568,NaN,NaN


,ratio_avoid
